# PV segmentation

Comparison between models (YOLOdetect + SAM)

source (Huggingface): finloop/yolov8s-seg-solar-panels (aka Rzeszów model)

source (Huggingface): https://huggingface.co/spaces/ArielDrabkin/Solar-Panel-Detector

source (Huggingface): andrewgray11/autotrain-solar-panel-object-detection-50559120777

credits: https://blog.roboflow.com/how-to-use-yolov8-with-sam/ (Roboflow)

XI 25

*MD*

## libs

In [1]:
# %pip install numpy
# %pip install pandas
# %pip install ultralytics

In [2]:
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO, SAM
import torch

In [3]:
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda', index=0)

## defs

### variants

In [4]:
sam = SAM("sam_b.pt")

In [5]:
models = {
    'finloop': 'best.pt',
    'ariel': 'final-mosaic-augmentation.pt',
    'pools': 'solarpanels_pools_yolov8l-p2_1024_v1.pt'
}

In [6]:
datasets = {
    'pilot': 'pilotPV_panels.v1i.yolov8-obb/test/images/*.jpg',
    'rzeszow_test': 'rzeszowSolar panels seg.v2i.yolov8-obb/test/images/*.jpg',
    # 'rzeszow_train': 'rzeszowSolar panels seg.v2i.yolov8-obb/train/images/*.jpg',
    'rzeszow_valid': 'rzeszowSolar panels seg.v2i.yolov8-obb/valid/images/*.jpg',
    'synth_test': 'auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/*.jpg',
    'synth_train': 'auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/*.jpg',
    'synth_valid': 'auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/*.jpg'
}

In [7]:
datasets = {
    'pilot': 'pilotPV_panels.v1i.yolov8-obb/test/images/*.jpg'}

### segment analysis

In [8]:
def sum_pv_segments(pth, model, nazwa="no_info_run", print_info=False, disp_img=False, display_coef=100):
    pv_area = 0
    yolo_results = model(pth, save=print_info, name=nazwa, stream=True, device=dev, verbose=print_info)
    for i, res in enumerate(yolo_results):
        dsp = disp_img and i % display_coef == 0 # limit
        ppth = res.path # nicely conveyed
        img_w, img_h = res.orig_shape # although YOLO reshapes img when necessary, SAM masks match org img
        if dsp:
            image = cv2.cvtColor(cv2.imread(ppth), cv2.COLOR_BGR2RGB)
            image = torch.tensor(image, device=dev)
        if res is not None and res.boxes is not None and res.boxes.xyxy is not None and len(res.boxes.xyxy > 0): # null-len res.boxes.xyxy for no PV
            sam_results = sam.predict(source=ppth, bboxes=res.boxes.xyxy)
            if sam_results is not None and sam_results[0] is not None and sam_results[0].masks is not None and sam_results[0].masks.data is not None:
                binary_mask = torch.where(sam_results[0].masks.data == True, 1, 0)
                mask_sum = binary_mask.sum(axis=0).data
                mask_sum_damped = torch.where(mask_sum >= 1, 1, 0) # actually some pxs covered multiple times - same object
                if dsp:
                    bcg_white = torch.ones_like(image)*255
                    new_image = bcg_white * (1 - mask_sum_damped[..., torch.newaxis]) + image * mask_sum_damped[..., torch.newaxis]
                    plt.imshow(new_image.reshape((img_w, img_h, 3)).cpu())
                    plt.title(f"Masked PVs in {ppth[ppth.rfind('/'):]}")
                    plt.axis('off')
                    plt.show()
                    # cv2.imwrite('c.png', new_image.reshape((img_w, img_h, 3)).cpu().numpy())
                pv_area += mask_sum_damped.sum().div(img_w*img_h) # percentage
                # print('mask sums', mask_sum.sum(), mask_sum_damped.sum())
                if print_info:
                    print(i, pv_area.item())
        if dsp:
            plt.imshow(image.cpu())
            plt.title(f"base img {ppth[ppth.rfind('/'):]}")
            plt.axis('off')
            plt.show()
    return pv_area

## run

In [9]:
for model_key, model in models.items():
    model = YOLO(model)
    for data_key, dataset in datasets.items():
        area = sum_pv_segments(dataset, model=model, nazwa=data_key, print_info=False)
        with open('sam_comparison.csv', 'a') as f:
            f.write(f'{data_key},{model_key},{area}\n')


image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-291-_png.rf.c105b47207c9dc4203900bab38f73c89.jpg: 1024x1024 1 0, 1 1, 1 2, 1221.5ms
Speed: 6.9ms preprocess, 1221.5ms inference, 15.6ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-293-_png.rf.6501de77787921a2b0c0a3fac3ff7882.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1197.6ms
Speed: 5.0ms preprocess, 1197.6ms inference, 1.3ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8-obb/test/images/Zrzut-ekranu-294-_png.rf.ec19d20eeaa7c3a1ea9c829158bbebdc.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1117.7ms
Speed: 4.2ms preprocess, 1117.7ms inference, 1.6ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /home/marekd6/ZPB/modelling/modelling/pilotPV_panels.v1i.yolov8